In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

In [ ]:
install.packages(c("tableone", "Matching", "ipw", "survey", "MatchIt", "sandwich", "pROC", "sjPlot"))

In [ ]:
library(tableone); library(Matching); library(ipw); library(ipw); library(MatchIt); library(sandwich)
library(pROC)

In [ ]:
library(dplyr)

In [ ]:
data(lalonde, package = 'MatchIt')

Fit a propensity score model.

Use a logistic regression model, where the outcome is treatment.

Include the 8 confounding variables in the model as predictors, with no interaction terms or non-linear terms (such as squared terms). Obtain the propensity score for each subject. 

Next, obtain the inverse probability of treatment weights for each subject.

In [ ]:
str(lalonde)

### Propensity score model

In [ ]:
psmodel <- glm(treat ~ age+educ+black+hispan+married+nodegree+re74+re75, data=lalonde, family=binomial(link='logit'))
pscore <- predict(psmodel, type='response')
lalonde$pscore <- pscore
roc(lalonde$treat, pscore)

### Making weights

In [ ]:
library(survey)

In [ ]:
xvars <- c('age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78')

In [ ]:
weight <- ifelse(lalonde$treat == 1, 1/(pscore), 1/(1-pscore))
weighteddata <- svydesign(ids = ~1, data=lalonde, weights = ~weight)
# weighted table 1
weightedtable <- svyCreateTableOne(xvars, strata='treat', data=weighteddata, test=FALSE)

In [ ]:
print(weightedtable0, smd=TRUE)

In [ ]:
print(weightedtable, smd=TRUE)

Reproducing age manually

In [ ]:
dfr <- data.frame(lalonde)
dfr$w <- weight
weighted.mean(dfr[dfr$treat==1, 'age'], dfr[dfr$treat==1, c('w')]);
weighted.mean(dfr[dfr$treat==0, 'age'], dfr[dfr$treat==0, c('w')])

#### q1

What are the minimum and maximum weights?

In [ ]:
min(weight); max(weight)

> q1 (1.0091633986672, 40.0772930454117)

#### q2

Find the standardized differences for each confounder on the weighted (pseudo) population. What is the standardized difference for nodegree?

In [ ]:
print(weightedtable, smd=TRUE)

> q2: smd(nodegree) = 0.11

#### q3

Using IPTW, find the estimate and 95% confidence interval for the average causal effect. 

This can be obtained from svyglm

In [ ]:
ate_fit <- svyglm(re78~treat, design=weighteddata)
summary(ate_fit)

In [ ]:
m <- summary(ate_fit)
m$coefficients[2,1] + c(-1, 1)*m$coefficients[2,2]*qt(0.975, df=nrow(weighteddata) - 1, lower.tail=FALSE)

> q3: 224.7 (-1562.8, 2012.2)

#### q4

Now truncate the weights at the 1st and 99th percentiles. This can be done with the trunc=0.01 option in svyglm.

Note: `trunc=0.01` doesn't work

In [ ]:
weight <- ifelse(lalonde$treat == 1, 1/(lalonde$pscore), 1/(1-lalonde$pscore))
weight_supp <- quantile(weight,c(.01, .99))

In [ ]:
ddf <- data.frame(lalonde)
ddf$w <- weight

In [ ]:
trunc_ddf <- filter(ddf, w > weight_supp[1] & w < weight_supp[2])

In [ ]:
tr_weight <- ifelse(trunc_ddf$treat == 1, 1/(trunc_ddf$pscore), 1/(1-trunc_ddf$pscore))

tr_weighteddata <- svydesign(ids = ~1, data=trunc_ddf, weights = ~tr_weight)

tr_ate_fit <- svyglm(re78~treat, design=tr_weighteddata)

summary(tr_ate_fit)

In [ ]:
m <- summary(tr_ate_fit)
m$coefficients[2,1] + c(-1, 1)*m$coefficients[2,2]*qt(0.975, df=m$df.residual, lower.tail=FALSE)

> q4: 505.8 (-1143.8, 2155.4)